In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import tensorflow as tf
from cv2 import cv2
import sys
import os
import time
import math
import numpy as np
from tqdm import tqdm

In [3]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Mounted at /content/drive


In [4]:
cd /content/drive/My\ Drive/Deeppicar-v3/

/content/drive/My Drive/Deeppicar-v3


In [5]:
!git update-index --assume-unchanged ./Dataset/

Ignoring path Dataset/


# Preprocess dataset video to frames

In [6]:
# Config
NFRAMES = 1000
vid_dir = "./Dataset/"

img_height = 66
img_width =  200
img_channels = 3
input_shape = [ img_height, img_width, img_channels]

In [7]:
mkdir ./Dataset/train_images

In [8]:
# TODO: Do not hard code the number of dataset files and filename
for i in tqdm(range(11)):
  
  # Open the video file
  vid_path = f"out-video-{i}"
  vid_ext = ".avi"
  path = vid_dir + vid_path + vid_ext
  assert os.path.isfile(path)
  cap = cv2.VideoCapture(path)

  curFrame = 0

  while (cap.isOpened()):
    if curFrame < NFRAMES:
      cam_start = time.time()
      ret, img = cap.read()

      if cv2.waitKey(1) & 0xFF == ord('q'):
        break

      curFrame += 1
      filename = f"train_images/{vid_path}_frame_{curFrame}.jpg" 
      frame = img
      # print(vid_dir + filename)
      cv2.imwrite(vid_dir + filename, frame)

    else:
      break

cap.release()
cv2.destroyAllWindows()

100%|██████████| 11/11 [01:35<00:00,  8.67s/it]


# Load the images and categorize image

In [9]:
from glob import glob
import pandas as pd

all_csv_files = glob("./Dataset/*.csv")
df_dict = {}
print(all_csv_files)
for filepath in all_csv_files:
  df = pd.read_csv(filepath, index_col=None,header=0)
  filename = filepath.split("/")[-1]
  out_number = filename.split("-")[-1]
  out_number = int(out_number.split(".")[0])
  df_dict[out_number] = df

['./Dataset/out-key-3.csv', './Dataset/out-key-4.csv', './Dataset/out-key-5.csv', './Dataset/out-key-2.csv', './Dataset/out-key-10.csv', './Dataset/out-key-6.csv', './Dataset/out-key-7.csv', './Dataset/out-key-9.csv', './Dataset/out-key-8.csv', './Dataset/out-key-0.csv', './Dataset/out-key-1.csv']


In [10]:
df_dict.keys()

dict_keys([3, 4, 5, 2, 10, 6, 7, 9, 8, 0, 1])

Let's check out the dataset 

In [11]:
df = df_dict[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ts_micro  1000 non-null   int64  
 1   frame     1000 non-null   int64  
 2   wheel     1000 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 23.6 KB


In [13]:
df.wheel.unique()

array([ 0.        ,  0.52359878, -0.52359878])

In [14]:
def preprocess(img):
  assert img_channels == 3 # for now we expect a color image
  ratio = img_height / img_width
  y1, y2 = 350, 553
  w = (y2-y1) / ratio
  padding = int(round((img.shape[1] - w) / 2))
  img = cv2.resize(img, (img_width, img_height))
  img = img / 255.
  return img

In [15]:
from google.colab.patches import cv2_imshow
from keras.preprocessing import image
images = glob("./Dataset/train_images/*.jpg")
train_image_path = []
train_image = []
train_wheel_val = []

from PIL import Image 
for i in tqdm(range(len(images))):

  image_path = images[i]
  train_image_path.append(image_path)

  # img = image.load_img(image_path, target_size=input_shape)
  img = image.load_img(image_path)
  # display(img)
  # converting it to array
  img = image.img_to_array(img)
  # # normalizing the pixel value
  # img = img / 255
  # # appending the image to the train_image list
  img = preprocess(img)
  # print(im)

  train_image.append(img)

  out_number = images[i].split("-")[-1]
  out_number = int(out_number.split("_")[0])
  df = df_dict[out_number]

  frame_number = images[i].split("_")[-1]
  frame_number = int(frame_number.split(".")[0])
  df_row = df[ df['frame'] == frame_number ]
  data = df_row['wheel']
  train_wheel_val.append(float(data))


100%|██████████| 11000/11000 [00:45<00:00, 242.12it/s]


In [16]:
train_df = pd.DataFrame()
train_df['image_path'] = train_image_path
train_df['wheel'] = train_wheel_val
train_df['image'] = train_image
train_df.head()

,image_path,wheel,image
0,./Dataset/train_images/out-video-0_frame_1.jpg,0.0,"[[[0.010516934, 0.011942958, 0.0], [0.01051693..."
1,./Dataset/train_images/out-video-0_frame_2.jpg,0.0,"[[[0.009269163, 0.011942958, 0.0], [0.00926916..."
2,./Dataset/train_images/out-video-0_frame_3.jpg,0.0,"[[[0.01114082, 0.013814616, 0.0], [0.009269163..."
3,./Dataset/train_images/out-video-0_frame_4.jpg,0.0,"[[[0.011942958, 0.015864525, 0.0], [0.00953654..."
4,./Dataset/train_images/out-video-0_frame_5.jpg,0.0,"[[[0.011942958, 0.015864525, 0.0], [0.00953654..."


In [17]:
X = np.array(train_image)
X.shape

(11000, 66, 200, 3)

Currently there is 11000 images with size of (66,200,3)<br>
- img_height = 66
- img_width =  200
- img_channels = 3  (RGB color)

In [18]:
from sklearn.model_selection import train_test_split
y = train_df['wheel'] 

X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.7, 
                                                    random_state=7,
                                                    stratify=y)

In [19]:
train_df['wheel'].unique()

array([ 0.        ,  0.52359878, -0.52359878])

# Define model 

The overall model look like this<br>
<img src='https://drive.google.com/uc?export=view&id=1ly2QuhyzF_DFADSfMPplvPk8u0ZuIm1c'
width='300px' height='300x'> <br>
And from tensorflow-v1:<br>
<img src='https://drive.google.com/uc?export=view&id=1fq4k7SnE-O1o3gppNjw8SumixwnpHZ_0' >





## Layers

In [20]:
# To make sure there is no inqeuivalent output from tf.nn.conv2d and keras.layers.Conv2D
# ref:https://stackoverflow.com/questions/61087933/inequivalent-output-from-tf-nn-conv2d-and-keras-layers-conv2d
tf.keras.backend.set_floatx('float32')

# Model 5 conv 3fc

#Input layer
input_x = tf.keras.layers.Conv2D(filters=3, kernel_size=[5,5], input_shape=input_shape,activation='relu',  padding="same")
input_max = tf.keras.layers.MaxPooling2D(5,2)

# 1st convolution layer
h_conv1 = tf.keras.layers.Conv2D(filters=24, kernel_size=[5,5], input_shape= input_shape,activation='relu',  padding="same")
h_max1 = tf.keras.layers.MaxPooling2D(5,2)

# 2nd convolution layer
h_conv2 = tf.keras.layers.Conv2D(filters=36, kernel_size=[5,5], activation='relu', padding="same") 
h_max2 = tf.keras.layers.MaxPooling2D(5,2)

# 3rd convolution layer
h_conv3 = tf.keras.layers.Conv2D(filters=48, kernel_size=[5,5], activation='relu', padding="same") 
h_max3 = tf.keras.layers.MaxPooling2D(3,1)

# 4th convolution layer
h_conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=[3,3], activation='relu', padding="same") 
h_max4 = tf.keras.layers.MaxPooling2D(3,1)

# 5th convolution layer
h_conv5 = tf.keras.layers.Conv2D(filters=64, kernel_size=[3,3], activation='relu', padding="same") 
h_conv5_flat = tf.keras.layers.Flatten() #1152

#Fully connected layers
h_fc2 = tf.keras.layers.Dense(100, activation='relu')
h_fc3 = tf.keras.layers.Dense(50, activation='relu')
h_fc4 = tf.keras.layers.Dense(10, activation='relu')
# missing atan and multiply
y = tf.keras.layers.Dense(3, activation='softmax')
                

In [21]:
# https://stackoverflow.com/questions/42785026/tf-nn-conv2d-vs-tf-layers-conv2d
model = tf.keras.models.Sequential([
            input_x,
            input_max,
            h_conv1,
            h_max1 ,
            h_conv2,
            h_max2,
            h_conv3,
            h_max3,
            h_conv4,
            h_max4,
            h_conv5,
            h_conv5_flat,
            h_fc2,
            h_fc3,
            h_fc4,
            y,
])

In [22]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 66, 200, 3)        228       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 98, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 47, 24)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 22, 36)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         4

In [27]:
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
loss = MeanSquaredError()
opt = Adam(learning_rate= 1e-4)
model.compile(loss=loss, optimizer=opt, metrics=['acc'])

In [28]:
epoch_num = 10
model.fit(X_train, y_train, epochs=epoch_num, validation_data=(X_test, y_test), batch_size=128)

Epoch 1/10
26/26 [==============================] - 97s 4s/step - loss: 0.3761 - acc: 0.4133 - val_loss: 0.3723 - val_acc: 0.4055
Epoch 2/10
26/26 [==============================] - 96s 4s/step - loss: 0.3723 - acc: 0.4136 - val_loss: 0.3723 - val_acc: 0.4055
Epoch 3/10
26/26 [==============================] - 96s 4s/step - loss: 0.3726 - acc: 0.4051 - val_loss: 0.3723 - val_acc: 0.4055
Epoch 4/10
26/26 [==============================] - 96s 4s/step - loss: 0.3728 - acc: 0.3981 - val_loss: 0.3723 - val_acc: 0.4055
Epoch 5/10
26/26 [==============================] - 96s 4s/step - loss: 0.3727 - acc: 0.4084 - val_loss: 0.3723 - val_acc: 0.4055
Epoch 6/10
26/26 [==============================] - 96s 4s/step - loss: 0.3749 - acc: 0.4014 - val_loss: 0.3723 - val_acc: 0.4055
Epoch 7/10
26/26 [==============================] - 96s 4s/step - loss: 0.3704 - acc: 0.4064 - val_loss: 0.3723 - val_acc: 0.4055
Epoch 8/10
26/26 [==============================] - 95s 4s/step - loss: 0.3675 - acc: 0.40